In [10]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from time import sleep
import concurrent.futures
import csv

In [11]:
companies = [
    "東京エレクトロン株式会社",
    "ソニーグループ株式会社",
    "パナソニックホールディングス株式会社",
    "株式会社ディスコ",
    "株式会社SCREENセミコンダクターソリューションズ",
    "フジクラ株式会社",
    "ルネサスエレクトロニクス株式会社",
    "ミスミグループ本社株式会社",
    "旭化成株式会社",
    "日立製作所株式会社",
    "日本板硝子株式会社",
    "ウシオ電機株式会社",
    "オムロン株式会社",
    "株式会社ニコン",
    "日東電工株式会社",
    "アプライドマテリアルズジャパン株式会社",
    "住友ベークライト株式会社",
    "ホシデン株式会社",
    "住友化学株式会社",
    "ソシオネクスト株式会社",
    "住友電気工業株式会社",
    "ジャパンディスプレイ株式会社",
    "株式会社アルバック",
    "株式会社日立ハイテク",
    "日本電気硝子株式会社",
    "メイコー株式会社",
    "株式会社ニューフレアテクノロジー",
    "日本特殊陶業株式会社",
    "東レエンジニアリング株式会社",
    "セイコーエプソン株式会社",
    "村田製作所株式会社",
    "イビデン株式会社",
    "大日本印刷株式会社",
    "安川電機株式会社",
    "シャープ株式会社",
    "株式会社荏原製作所",
    "ダイヘン株式会社",
    "京セラ株式会社",
    "ヤマハ発動機株式会社",
    "信越化学工業株式会社",
    "株式会社東京精密",
    "浜松ホトニクス株式会社",
    "スタンレー電気株式会社",
    "マブチモーター株式会社",
    "富士電機株式会社",
    "豊田合成株式会社",
    "三菱マテリアル株式会社",
    "新光電気工業株式会社",
    "ローム株式会社",
    "アズビル株式会社",
    "ミネベアミツミ株式会社",
    "日本電子株式会社",
    "古河電気工業株式会社",
    "ダイトロン株式会社",
    "三菱電機株式会社",
    "日新電機株式会社",
    "アイカ工業株式会社",
    "サンケン電気株式会社",
    "明電舎株式会社",
    "TOWA株式会社",
    "CKD株式会社",
    "芝浦メカトロニクス株式会社",
    "日本航空電子工業株式会社",
    "キヤノンアネルバ株式会社",
    "シチズン時計株式会社",
    "アルプスアルパイン株式会社",
    "ノリタケカンパニーリミテド株式会社",
    "日本電子株式会社",
    "ヒロセ電機株式会社",
    "住友精密工業株式会社",
    "日本ケミコン株式会社",
    "ニデック株式会社",
    "キヤノンマシナリー株式会社",
    "太陽誘電株式会社",
    "ニチコン株式会社",
    "富士通株式会社",
    "三井ハイテック株式会社",
    "キヤノン株式会社",
    "デンカ株式会社",
    "株式会社高田工業所"
]

In [12]:
# スクレイピングの結果を格納するための辞書
results = {}

# スクレイピング関数
def fetch_company_data(search_term):
    search_url = 'https://en-hyouban.com/search/'
    params = {'SearchWords': search_term}
    response = requests.get(search_url, params=params)
    if response.ok:
        return response.text
    return None

def find_company_profile_page(html_content, company_name):
    soup = BeautifulSoup(html_content, 'html.parser')
    for a in soup.select('a'):
        if company_name in a.text:
            return urljoin('https://en-hyouban.com', a['href'])
    return None

def fetch_company_profile_page(url):
    response = requests.get(url)
    if response.ok:
        return response.text
    return None

def parse_company_profile_page(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)') else '情報なし'
    overtime = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(2)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(2)') else '情報なし'
    return {
        'salary': salary,
        'overtime': overtime
    }


def parse_company_profile_page(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)') else '情報なし'
    
    # 使用する警告を修正: 'text' -> 'string'
    overtime_section = soup.find('div', string=lambda t: t and '残業時間' in t)
    if overtime_section:
        overtime = overtime_section.find_next('span', class_='font-change-30').get_text(strip=True) if overtime_section.find_next('span', class_='font-change-30') else '情報なし'
    else:
        overtime = '情報なし'

    return {
        'salary': salary,
        'overtime': overtime
    }


def parse_company_profile_page(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)') else '情報なし'
    
    # 残業時間情報を含むセクションを探す
    overtime_section = soup.find('div', class_='col-6 px-0 text-center pt-1')
    if overtime_section:
        # 残業時間の値を含むspan要素を探す
        overtime_span = overtime_section.find('span', class_='font-change-30')
        overtime = overtime_span.get_text(strip=True) if overtime_span else '情報なし'
    else:
        overtime = '情報なし'

    return {
        'salary': salary,
        'overtime': overtime
    }


In [13]:
def fetch_salary_details(company_id):
    details_url = f'https://en-hyouban.com/company/{company_id}/salary/'
    response = requests.get(details_url)
    if response.ok:
        return response.text
    return None

def parse_salary_details(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary_details = {}
    rows = soup.select('#salary_information_by_age tr')
    for row in rows:
        age_group = row.select_one('.job-type')
        if age_group:
            age_text = age_group.get_text(strip=True)
            # Replace '以上' with '_up' and other age ranges with '_' for consistency
            age_key = age_text.replace('~', '_').replace('歳', '').replace('以上', '_up')
            salary_key = f'salary_{age_key}'  # Create a key like 'salary_25_29'
            salary_value = row.select_one('.salary-font-size').get_text(strip=True) if row.select_one('.salary-font-size') else '情報なし'
            salary_details[salary_key] = salary_value
    return salary_details




def fetch_and_parse_company_data(company):
    search_page_content = fetch_company_data(company)
    if search_page_content:
        profile_page_url = find_company_profile_page(search_page_content, company)
        if profile_page_url:
            sleep(1)  # URLを見つけた後にサーバーの負荷を減らすために待機
            profile_page_content = fetch_company_profile_page(profile_page_url)
            if profile_page_content:
                company_data = parse_company_profile_page(profile_page_content)
                # Extract company ID from profile page URL
                company_id = profile_page_url.split('/')[-2]
                salary_details_content = fetch_salary_details(company_id)
                if salary_details_content:
                    salary_details = parse_salary_details(salary_details_content)
                    company_data.update(salary_details)
                return company, company_data
    return company, None


# 結果をCSVに保存する関数 (更新されたフィールド名)
def save_results_to_csv(results, filename=fr'C:\Users\100ca\Documents\PyCode\scraping\data\results.csv'):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['company', 'salary', 'overtime', 'salary_25_29', 'salary_30_34', 'salary_35_39', 'salary_40_44', 'salary_45_49', 'salary_50_54', 'salary_55_59', 'salary_60_up']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for company, data in results.items():
            if data:
                writer.writerow({**{'company': company}, **data})

# 並列処理で各企業に対してスクレイピングを実行
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_company = {executor.submit(fetch_and_parse_company_data, company): company for company in companies}
    results = {}
    for future in concurrent.futures.as_completed(future_to_company):
        company = future_to_company[future]
        try:
            data = future.result()
            results[data[0]] = data[1]
        except Exception as exc:
            print(f'{company} generated an exception: {exc}')

# 結果をCSVに保存
save_results_to_csv(results)


# 結果の出力 (更新されたフィールド名)
for company, data in results.items():
    if data:
        print(f"{company}: 年収={data.get('salary', '情報なし')}, 残業時間={data.get('overtime', '情報なし')}, " +
              f"25-29歳平均年収={data.get('salary_25_29', '情報なし')}, 30-34歳平均年収={data.get('salary_30_34', '情報なし')}, " +
              f"35-39歳平均年収={data.get('salary_35_39', '情報なし')}, 40-44歳平均年収={data.get('salary_40_44', '情報なし')}, " +
              f"45-49歳平均年収={data.get('salary_45_49', '情報なし')}, 50-54歳平均年収={data.get('salary_50_54', '情報なし')}, " +
              f"55-59歳平均年収={data.get('salary_55_59', '情報なし')}, 60歳以上平均年収={data.get('salary_60_up', '情報なし')}")

株式会社ディスコ: 年収=760, 残業時間=37, 25-29歳平均年収=697, 30-34歳平均年収=697, 35-39歳平均年収=情報なし, 40-44歳平均年収=情報なし, 45-49歳平均年収=787, 50-54歳平均年収=情報なし, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
株式会社SCREENセミコンダクターソリューションズ: 年収=746, 残業時間=28, 25-29歳平均年収=情報なし, 30-34歳平均年収=658, 35-39歳平均年収=情報なし, 40-44歳平均年収=情報なし, 45-49歳平均年収=情報なし, 50-54歳平均年収=情報なし, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
ソニーグループ株式会社: 年収=877, 残業時間=34, 25-29歳平均年収=645, 30-34歳平均年収=704, 35-39歳平均年収=764, 40-44歳平均年収=853, 45-49歳平均年収=1304, 50-54歳平均年収=939, 55-59歳平均年収=1141, 60歳以上平均年収=1224
東京エレクトロン株式会社: 年収=940, 残業時間=38, 25-29歳平均年収=524, 30-34歳平均年収=707, 35-39歳平均年収=780, 40-44歳平均年収=情報なし, 45-49歳平均年収=1056, 50-54歳平均年収=情報なし, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
パナソニックホールディングス株式会社: 年収=792, 残業時間=29, 25-29歳平均年収=489, 30-34歳平均年収=716, 35-39歳平均年収=778, 40-44歳平均年収=855, 45-49歳平均年収=843, 50-54歳平均年収=1086, 55-59歳平均年収=936, 60歳以上平均年収=1123
ルネサスエレクトロニクス株式会社: 年収=728, 残業時間=34, 25-29歳平均年収=631, 30-34歳平均年収=529, 35-39歳平均年収=638, 40-44歳平均年収=740, 45-49歳平均年収=722, 50-54歳平均年収=863, 55-59歳平均年収=778, 60歳以上平均年収=767
旭化成株式会社: 年収=727, 

In [14]:
data

{'salary': '367',
 'overtime': '40',
 'salary_25_29': '359',
 'salary_30_34': '382',
 'salary_35_39': '450'}